In [8]:
import pandas as pd
import numpy as np
import seaborn as sn
from suntime import Sun
import pytz
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna

# Helper Functions

In [11]:
def datetime_index(dataframe):
    dataframe.columns = dataframe.columns.str.replace(' ', '')
    dataframe['Datetime'] = pd.to_datetime(dataframe['DateTime'])
    dataframe = dataframe.set_index(['Datetime'])
    del dataframe['DateTime']
    return dataframe

def fill_na(dataframe):
    dataframe['WWCode'] = dataframe['WWCode'].fillna(0).astype(int)
    return dataframe

def day_night(dataframe):

    coordinates = [40.239, 33.029] # Ankara
    dataframe['datetime_col'] = dataframe.index
    
    dataframe["Date"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.date.apply(str)
    dataframe["Hour"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.hour
    dataframe["Month"] = dataframe['datetime_col'].apply(pd.to_datetime).dt.month
    
    sun = Sun(coordinates[0], coordinates[1])
    tz =pytz.timezone('Europe/Istanbul')
    dataframe["Day"]= dataframe[["Date","Hour"]].apply(lambda x : sun.get_local_sunrise_time(pd.to_datetime(x["Date"]).
                                                                date(),local_time_zone=tz).
                                         hour <= x["Hour"] <= sun.get_local_sunset_time(pd.to_datetime(x["Date"]).
                                                                date(),local_time_zone=tz).hour ,axis=1)
    dataframe = dataframe.drop(['Date', 'datetime_col', 'Hour'], axis = 1)
    return dataframe

def wwcode_encoder(dataframe):
    for index, row in dataframe.iterrows():
        if row['WWCode'] < 10:
            dataframe.loc[index, 'WWCode'] = 1
        elif row['WWCode'] < 20:
            dataframe.loc[index, 'WWCode'] = 2
        elif row['WWCode'] < 30:
            dataframe.loc[index, 'WWCode'] = 3
        elif row['WWCode'] < 40:
            dataframe.loc[index, 'WWCode'] = 4
        elif row['WWCode'] < 50:
            dataframe.loc[index, 'WWCode'] = 5
        elif row['WWCode'] < 60:
            dataframe.loc[index, 'WWCode'] = 6
        elif row['WWCode'] < 70:
            dataframe.loc[index, 'WWCode'] = 7
        elif row['WWCode'] < 80:
            dataframe.loc[index, 'WWCode'] = 8
        elif row['WWCode'] < 90:
            dataframe.loc[index, 'WWCode'] = 9
        elif row['WWCode'] < 100:
            dataframe.loc[index, 'WWCode'] = 10
    return dataframe
    
    
def season_encoder(dataframe):
    dataframe['Season'] = 0
    for index, row in dataframe.iterrows():
        if row['Month'] < 3 or row['Month'] == 12:
            dataframe.loc[index, 'Season'] = 1
        elif row['Month'] < 6:
            dataframe.loc[index, 'Season'] = 2
        elif row['Month'] < 9:
            dataframe.loc[index, 'Season'] = 3
        elif row['Month'] < 12:
            dataframe.loc[index, 'Season'] = 4
    return dataframe

def fig_plot():
    plt.figure(figsize = (10,15))
    plt.scatter(Y_test, pred_val)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    
    
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    early_stopping_rounds = trial.suggest_int("early_stopping_rounds", 100, 500)
    n_estimators = trial.suggest_int("n_estimators", 0, 10000)
    
    model = XGBRegressor(
        random_state=50,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(
        X_train,
        Y_train,
        early_stopping_rounds=early_stopping_rounds,
        eval_set=[(X_test, Y_test)],
        verbose=1200,
    )
    preds_valid = model.predict(X_test)
    rmse = mean_squared_error(Y_test, preds_valid, squared=False)

    return rmse

# Data Extraction

In [3]:
submission = pd.read_csv('sample_submission.csv')
label = datetime_index(pd.read_csv('generation.csv', delimiter=';', decimal=','))
features = datetime_index(pd.read_csv('temperature.csv', delimiter=';', decimal=','))

In [4]:
x_train  = fill_na(features.loc['2019-01-01':'2021-11-30'])
x_test = day_night(fill_na(features.loc['2021-12-01':'2021-12-31']))
y_train  = label.loc['2019-01-01':'2021-11-30']

In [5]:
df_train = day_night(pd.concat([ x_train, y_train], axis=1))

### Preprocessing

In [6]:
df_train = wwcode_encoder(df_train)
df_train = season_encoder(df_train)

x_test = wwcode_encoder(x_test)
x_test = season_encoder(x_test)

In [18]:
df_train.drop(['ComfortTemperature'], axis=1, inplace=True)
x_test.drop(['ComfortTemperature'], axis=1, inplace=True)

In [19]:
# df_train = pd.get_dummies(df_train, columns = ['WWCode'])
# x_test = pd.get_dummies(x_test, columns = ['WWCode'])

In [22]:
X_train = df_train.loc['2019-01-01':'2021-10-31'].drop(['Generation'], axis=1)
X_test = df_train.loc['2021-10-01':'2021-11-30'].drop(['Generation'], axis=1)
Y_train = df_train.Generation.loc['2019-01-01':'2021-10-31']
Y_test = df_train.Generation.loc['2021-10-01':'2021-11-30']

In [20]:
# X_train = df_train.loc['2019-01-01':'2021-04-30'].drop(['Generation'], axis=1)
# X_test = df_train.loc['2021-05-01':'2021-11-30'].drop(['Generation'], axis=1)
# Y_train = df_train.Generation.loc['2019-01-01':'2021-04-30']
# Y_test = df_train.Generation.loc['2021-05-01':'2021-11-30']

In [23]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2022-02-28 15:15:45,226] A new study created in memory with name: no-name-0c33d3cb-75ef-4b01-802f-b90310760afd


[0]	validation_0-rmse:148.85928
[1200]	validation_0-rmse:84.69991
[2400]	validation_0-rmse:83.83836
[3600]	validation_0-rmse:83.45020
[4218]	validation_0-rmse:83.44394


[I 2022-02-28 15:15:58,335] Trial 0 finished with value: 83.39278238074412 and parameters: {'learning_rate': 0.021839067106067006, 'reg_lambda': 0.025123627606067366, 'reg_alpha': 1.8097411492019137e-07, 'subsample': 0.4743245973635447, 'colsample_bytree': 0.6160376594632305, 'max_depth': 1, 'early_stopping_rounds': 326, 'n_estimators': 4219}. Best is trial 0 with value: 83.39278238074412.


[0]	validation_0-rmse:140.99083
[1200]	validation_0-rmse:69.90490
[2400]	validation_0-rmse:68.32053
[2884]	validation_0-rmse:68.24579


[I 2022-02-28 15:16:15,574] Trial 1 finished with value: 68.2017920938956 and parameters: {'learning_rate': 0.197963121242954, 'reg_lambda': 1.3394808254809756e-07, 'reg_alpha': 0.00045835482417194755, 'subsample': 0.8649233686254489, 'colsample_bytree': 0.3060108894137349, 'max_depth': 5, 'early_stopping_rounds': 422, 'n_estimators': 6068}. Best is trial 1 with value: 68.2017920938956.


...

[I 2022-02-28 15:31:46,131] Trial 17 finished with value: 48.39962606715297 and parameters: {'learning_rate': 0.028609671871781713, 'reg_lambda': 0.9664087883379908, 'reg_alpha': 0.4887708925164263, 'subsample': 0.8832905855360118, 'colsample_bytree': 0.6540766973770097, 'max_depth': 7, 'early_stopping_rounds': 383, 'n_estimators': 9257}. Best is trial 11 with value: 47.295487858165096.


[0]	validation_0-rmse:145.68042
[160]	validation_0-rmse:74.95637


[I 2022-02-28 15:31:47,053] Trial 18 finished with value: 74.9563698434117 and parameters: {'learning_rate': 0.048151715173983055, 'reg_lambda': 0.00010371945191753496, 'reg_alpha': 8.51326022805378e-06, 'subsample': 0.633733969137377, 'colsample_bytree': 0.9647255328522821, 'max_depth': 4, 'early_stopping_rounds': 458, 'n_estimators': 161}. Best is trial 11 with value: 47.295487858165096.


[0]	validation_0-rmse:149.32550
[1200]	validation_0-rmse:69.09431
[2400]	validation_0-rmse:66.84476
[3600]	validation_0-rmse:65.19320
[4800]	validation_0-rmse:63.75746
[6000]	validation_0-rmse:62.51363
[7200]	validation_0-rmse:61.54985
[7542]	validation_0-rmse:61.26482


[I 2022-02-28 15:33:01,256] Trial 19 finished with value: 61.26409149103658 and parameters: {'learning_rate': 0.014049765408579454, 'reg_lambda': 4.039196563205417, 'reg_alpha': 0.00019373719188297102, 'subsample': 0.8083122394655102, 'colsample_bytree': 0.4852519853899815, 'max_depth': 6, 'early_stopping_rounds': 207, 'n_estimators': 7543}. Best is trial 11 with value: 47.295487858165096.


In [24]:
optuna_params = study.best_params
optuna_params

{'learning_rate': 0.05374230445285003,
 'reg_lambda': 0.38343772693998585,
 'reg_alpha': 0.01466583197618077,
 'subsample': 0.9790394943156467,
 'colsample_bytree': 0.7749376098752156,
 'max_depth': 7,
 'early_stopping_rounds': 493,
 'n_estimators': 9963}

In [25]:
model = XGBRegressor(
    random_state=50,
    tree_method="gpu_hist",
    gpu_id=0,
    predictor="gpu_predictor",
    **optuna_params
)
model.fit(
    X_train,
    Y_train,
    early_stopping_rounds=optuna_params['early_stopping_rounds'],
    eval_set=[(X_test, Y_test)],
    verbose=1000,
)

[15:33:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:146.57371
[1000]	validation_0-rmse:57.30534
[2000]	validation_0-rmse:51.96480
[3000]	validation_0-rmse:49.16534
[4000]	validation_0-rmse:48.05273
[5000]	validation_0-rmse:47.56355
[6000]	validation_0-rmse:47.36774
[7000]	validation_0-rmse:47.33351
[7849]	validation_0-rmse:47.32890


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7749376098752156,
             early_stopping_rounds=493, enable_categorical=False, gamma=0,
             gpu_id=0, importance_type=None, interaction_constraints='',
             learning_rate=0.05374230445285003, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=9963, n_jobs=8, num_parallel_tree=1,
             predictor='gpu_predictor', random_state=50,
             reg_alpha=0.01466583197618077, reg_lambda=0.38343772693998585,
             scale_pos_weight=1, subsample=0.9790394943156467,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [26]:
pred_val_optuna = model.predict(X_test)
pred_val_optuna = pd.DataFrame(pred_val_optuna, index=X_test.index, columns=['pred'])

In [27]:
pred_val_optuna = day_night(pred_val_optuna)
pred_val_optuna.pred[pred_val_optuna.Day == False] = 0
pred_val_optuna = pred_val_optuna.drop(['Month', 'Day'], axis=1)
pred_val_optuna

C:\Users\asus\AppData\Local\Temp/ipykernel_7036/3066051035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_val_optuna.pred[pred_val_optuna.Day == False] = 0


,pred
Datetime,
2021-10-01 00:00:00,0.0
2021-10-01 01:00:00,0.0
2021-10-01 02:00:00,0.0
2021-10-01 03:00:00,0.0
2021-10-01 04:00:00,0.0
...,...
2021-11-30 19:00:00,0.0
2021-11-30 20:00:00,0.0
2021-11-30 21:00:00,0.0


In [28]:
rmse_optuna = mean_squared_error(Y_test, pred_val_optuna, squared=False)
rmse_optuna

46.65403921577873